In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import tensorflow
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


train = pd.read_csv('/content/drive/MyDrive/data/손글씨/train/train_data.csv')

images = []
counts = 0

for image_name in train['file_name']:
    image = np.array(Image.open('/content/drive/MyDrive/data/손글씨/train/' + image_name)).flatten()
    images.append(image)
    counts+=1
    if counts%500==0:
      print(counts)

image_full = pd.DataFrame(images)
image_full['label'] = train['label']


data_x = image_full.iloc[:,:-1]
data_x = data_x/255.0
data_x = data_x.values.reshape(-1,28,28,1)
data_y = to_categorical(train['label'], num_classes = 10)

x_train, x_val, y_train, y_val = train_test_split(data_x, data_y, test_size = .2, random_state = 1234, stratify = data_y)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

augmentation = ImageDataGenerator(featurewise_center = False,
                                 samplewise_center = False,
                                 featurewise_std_normalization = False,
                                 samplewise_std_normalization = False,
                                 zca_whitening = False,
                                 rotation_range = 15,
                                 zoom_range = 0.15,
                                 width_shift_range = 0.15,
                                 height_shift_range = 0.15,  
                                 horizontal_flip = False,
                                 vertical_flip = False)

augmentation.fit(x_train)

In [ ]:
import tensorflow
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=64, input_shape=(28,28,1), padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tensorflow.keras.layers.MaxPool2D(pool_size=(2,2)),
    tensorflow.keras.layers.Dropout(0.2),

    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tensorflow.keras.layers.MaxPool2D(pool_size=(2,2)),
    tensorflow.keras.layers.Dropout(0.2),

    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tensorflow.keras.layers.MaxPool2D(pool_size=(2,2)),
    tensorflow.keras.layers.Dropout(0.2),

    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(units=256, activation='relu'),
    tensorflow.keras.layers.Dense(units=64, activation='relu'),
    tensorflow.keras.layers.Dense(units=32, activation='relu'),
    tensorflow.keras.layers.Dropout(0.2),
    tensorflow.keras.layers.Dense(units = 10, activation='softmax')
])
optimizer = tensorflow.keras.optimizers.RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08, decay = 0.0 )
model.compile(optimizer = optimizer, 
              loss = "categorical_crossentropy", 
              metrics = ["accuracy"])


call_back = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.0001)

In [ ]:
history = model.fit_generator(augmentation.flow(x_train, y_train, batch_size = 64),
                              epochs = 400, 
                              validation_data = (x_val, y_val),
                              verbose = 1,
                              steps_per_epoch=x_train.shape[0] // 64,  
                              callbacks=[call_back])

Epoch 1/400
62/62 [==============================] - 15s 54ms/step - loss: 2.2949 - accuracy: 0.1151 - val_loss: 2.0907 - val_accuracy: 0.2560 - lr: 0.0010
Epoch 2/400
62/62 [==============================] - 3s 45ms/step - loss: 1.9329 - accuracy: 0.3143 - val_loss: 0.9751 - val_accuracy: 0.6790 - lr: 0.0010
Epoch 3/400
62/62 [==============================] - 3s 46ms/step - loss: 1.4046 - accuracy: 0.5376 - val_loss: 0.4953 - val_accuracy: 0.8550 - lr: 0.0010
Epoch 4/400
62/62 [==============================] - 3s 46ms/step - loss: 0.9167 - accuracy: 0.7083 - val_loss: 0.3225 - val_accuracy: 0.9110 - lr: 0.0010
Epoch 5/400
62/62 [==============================] - 3s 46ms/step - loss: 0.6398 - accuracy: 0.8084 - val_loss: 0.6156 - val_accuracy: 0.8220 - lr: 0.0010
Epoch 6/400
62/62 [==============================] - 3s 46ms/step - loss: 0.5283 - accuracy: 0.8473 - val_loss: 0.1448 - val_accuracy: 0.9590 - lr: 0.0010
Epoch 7/400
62/62 [==============================] - 3s 47ms/step - l

In [ ]:
model.save('/content/drive/MyDrive/data/손글씨/model_image_aug6.h5')

In [ ]:
from tensorflow.keras.models import load_model



model = load_model('/content/drive/MyDrive/data/손글씨/model_image_aug6.h5')

test = pd.read_csv('/content/drive/MyDrive/data/손글씨/test/test_data.csv')

test_images = []
counts = 0

for image_name in test['file_name']:
    image = np.array(Image.open('/content/drive/MyDrive/data/손글씨/test/' + image_name)).flatten()
    test_images.append(image)
    counts+=1
    if counts%500==0:
      print(counts)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [ ]:
image_full_test = pd.DataFrame(test_images)

data_x = image_full_test/255.0
data_x = data_x.values.reshape(-1,28,28,1)

In [ ]:
pred = model.predict(data_x)
predict = np.argmax(pred, axis = 1)


In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/data/손글씨/sample_submission.csv')
sub.head()

,file_name,label
0,idx0001.png,-1
1,idx0002.png,-1
2,idx0003.png,-1
3,idx0004.png,-1
4,idx0005.png,-1


In [ ]:
sub['label'] = predict
sub.head()

,file_name,label
0,idx0001.png,1
1,idx0002.png,0
2,idx0003.png,8
3,idx0004.png,3
4,idx0005.png,5


In [ ]:
sub.to_csv('/content/drive/MyDrive/data/손글씨/sub_image_aug6.csv', index = False)

In [ ]:
# sub_image_aug기준 0.895 epoch 100
# sub_image_aug2기준 0.908 epoch 100 + 150
# sub_image_aug3기준 0.91 epoch 400

# sub_image_aug6기준 0.93 원래 레이어 + conv층 추가(필터 32) epoch 400